In [1]:
import mne
import pandas as pd
import os

import matplotlib.pyplot as plt


import numpy as np
import platform

import sys
if platform.system()=='Darwin':
    os.chdir('/Volumes/BBC/BBC/WP1/data/EEG/tsk/')
    sys.path.append('/Users/leupinv/BBC/WP1/data/Code/python/BBC')
    #if this doesn't work pound line right above this, restart the kernel, rerun the cell.
    #Then uncomment the same line and rerun

else:
    #os.chdir('Z:/BBC/WP1/data/EEG/tsk')
    sys.path.append('C:/Users/Vivi/switchdrive/BBC/WP1/data/Code/python/BBC')
    os.chdir('Z:/BBC/WP1/data/EEG/tsk')

from base.files_in_out import GetFiles
from stats.stats_helper import filter_list, tTest_ana, filter_list_equal
import stats.stats_helper as s_hp


C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
C:\Users\Vivi\anaconda3\envs\mne\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
%load_ext autoreload
%autoreload 2
import matplotlib as mpl
mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False
plt.style.use('default')

mpl.rcParams['svg.fonttype']='none'
mpl.rcParams['text.usetex']=False

In [3]:
g_num=None
eeg_format='-ave.fif'
eeg_exp='tsk'
datafolder='ana/MNE/evo_list'


#files = getListOfFiles(datafolder,g_num)

files = GetFiles(datafolder,g_num=g_num,eeg_format=eeg_format)

evo_files=files.fflist

# Correlation RT and ERPs


## Load df with RT

In [4]:
# Load RT df
path='ana/behavioral/df_o_cleantrials.feather'
RT_df=pd.read_feather(path)

In [5]:
# fitler only normal correct trials and end of the diastole
RT_df=RT_df[RT_df['difficulty']== 'normal']

RT_df=RT_df[RT_df['sys_mask']== 1]
RT_df=RT_df[RT_df['accuracy']== 'correct']

## Define parameters for ERPs


In [6]:
picks_P1=['D32','A11','A12','A13','A14','A24','A25','A26','A27','B8','B9','B10']


picks_P2=['A13','A14','A15','A23','A24','A25','A26','A27','A28']

picks_VAN=['D32','A9','A10','A11','A12','A16','A15','A14','A13',
           'A22','A23','A24','A25','A29','A28','A27','A26','B6','B7','B8','B9','B10']

picks_P3a=['C11','C12','C13','C14','C15','C18','C19','C20','C21','C22','C25','C26','C27','C28','C32','D4']

picks_P3b=['A2','A3','A4','A19','A20','A21','A22','A32','A31','A30','A29','A5','A18',
          'A17','A16','A8','A9','A6','A7','D29','D30','D15','D16','D17','D28','D27']

picks_dict=dict(p1=picks_P1,p2=picks_P2,VAN=picks_VAN,p3a=picks_P3a,p3b=picks_P3b)
time_int_dict= dict(p1=[0.09,0.12],p2=[0.14,0.16],VAN=[0.24,0.32 ],p3a=[0.32,0.36],p3b=[0.4,0.5])

## Compute DF from ERPs
Load erps, extract the potentials, prepare RT df to correspond with ERPs. Combine the two and thens save

In [7]:
phy_cond_list=['sys','dia','inh','exh']
awa_cond_list=['aware','unaware']


for phy_cond in phy_cond_list:
    for awa_cond in awa_cond_list:
        # load ERP for condition
        cond=mne.read_evokeds(filter_list_equal(evo_files,f'tsk_o_maskON_correct_cfa_vep_norm_{awa_cond}_{phy_cond}_list'))
        # uses function that extract the avg potentials from components and time and returns df
        pd_df=s_hp.extract_components_corr(cond,picks_dict,time_int_dict)
        # Filter RT df and create df with averaged RT for each subject
        pivot_RT_df= s_hp.filter_RT_df_corr(RT_df.copy(),awa_cond,phy_cond)
        # Combine ERP and RT df to correlate
        final_df=pivot_RT_df.merge(pd_df,on='g_num')
        #assign variable with condition in case of future merges
        final_df['cond']=f'{awa_cond}_{phy_cond}'
        #save df
        path=f'ana/MNE/components_avg/amplitudes_RT_{awa_cond}_{phy_cond}.csv'
        final_df.to_csv(path)



ana/MNE/evo_list/maskON/mistake/nc/xns/norm/maskON_mistake_nc_xns_norm_unaware_sys_list-ave.fif
Reading ana/MNE/evo_list/maskON/correct/cfa/vep/norm/tsk_o_maskON_correct_cfa_vep_norm_aware_sys_list-ave.fif ...
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =    -199.22 ...    1000.00 ms (0.42 × vep/normal/correct/aware/sys/exh + 0.58 × vep/normal/correct/aware/sys/inh
 G_n= g01)
        0 CTF compensation matrices available
        nave = 95 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
No baseline correction applied
    Read a total of 1 projection items:
        Average EEG reference (1 x 128) active
    Found the data of interest:
        t =    -199.22 ...    1000.00 ms (0.40 × vep/normal/correct/aware/sys/exh + 0.60 × vep/normal/correct/aware/sys/inh
 G_n= g03)
        0 CTF compensation matrices available
        nave = 121 - aspect type = 100
Projections

## Reload each df with amplitude and RT and compute correlations
combine DF with all correlations and p values and save those
compute FDR correction over pval vectors and save corrected df

In [74]:
corr_df=pd.DataFrame()
p_val_df=pd.DataFrame()
methods=['pearson','spearman']
for method in methods:
    corr_df=pd.DataFrame()
    p_val_df=pd.DataFrame()
    for phy_cond in phy_cond_list:
        for awa_cond in awa_cond_list:
            # read df
            path=f'ana/MNE/components_avg/amplitudes_RT_{awa_cond}_{phy_cond}.csv'
            df=pd.read_csv(path)
            # set index to gnum
            df=df.set_index('g_num').drop(columns=['Unnamed: 0','cond'])
            # compute corr using pearson
            correlations_df=df.corr(method=method)

            if method=='pearson':
                p_df=s_hp.calculate_pvalues_pearson(df)
            elif method=='spearman':
                p_df=s_hp.calculate_pvalues_spearman(df)
            else:
                raise ('Mistake in method')

            RT_row_corr=s_hp.filter_corr_df(correlations_df,awa_cond,phy_cond)
            RT_row_pval=s_hp.filter_corr_df(p_df,awa_cond,phy_cond)
            #append series as a new row to pd df
            corr_df=pd.concat([corr_df,RT_row_corr])
            p_val_df=pd.concat([p_val_df,RT_row_pval])
            # drop RT column
            corr_df=corr_df.drop(columns='RT')
            p_val_df=p_val_df.drop(columns='RT')

    #save dfs
    path_corr=f'ana/MNE/components_avg/amplitudes_RT_corr_{method}.csv'
    path_pval=f'ana/MNE/components_avg/amplitudes_RT_pval_{method}.csv'
    corr_df.to_csv(path_corr)
    p_val_df.to_csv(path_pval)


In [67]:
p_val_df

,p1,p2,VAN,p3a,p3b
aware_sys,0.568,0.0445,0.6769,0.8751,0.4419
unaware_sys,0.7323,0.0466,0.5844,0.5144,0.601
aware_dia,0.5851,0.0423,0.4813,0.7018,0.3061
unaware_dia,0.7024,0.0234,0.4909,0.5831,0.7964
aware_inh,0.4161,0.0647,0.7671,0.8726,0.4871
unaware_inh,0.6025,0.059,0.6736,0.5675,0.8264
aware_exh,0.6601,0.027,0.4286,0.8177,0.3898
unaware_exh,0.6964,0.0204,0.4291,0.5185,0.5952
aware_sys,0.7534,0.039,0.6697,0.6731,0.1371
unaware_sys,0.6443,0.0354,0.9656,0.96,0.4494


In [101]:
method='spearman'
path_pval=f'ana/MNE/components_avg/amplitudes_RT_pval_{method}.csv'
df_pval=pd.read_csv(path_pval)
df_pval=df_pval.drop(columns='Unnamed: 0')
df_pval

,p1,p2,VAN,p3a,p3b
0,0.7534,0.0390,0.6697,0.6731,0.1371
1,0.6443,0.0354,0.9656,0.9600,0.4494
2,0.6275,0.0304,0.4311,0.4551,0.2014
3,0.8000,0.0147,0.6527,0.7445,0.2407
4,0.9117,0.0464,0.3340,0.8951,0.1479
5,0.7499,0.0439,0.6142,0.9935,0.3932
6,0.7802,0.0217,0.8272,0.4783,0.1444
7,0.7463,0.0176,0.9618,0.6561,0.2331


In [102]:
array=df_pval.to_numpy().flatten()

In [103]:
from statsmodels.stats.multitest import multipletests
correct_array=multipletests(array, method='fdr_bh')

In [104]:
corr_df=correct_array[1].reshape(8,5)

In [105]:
corr_df

array([[0.96969697, 0.232     , 0.96969697, 0.96969697, 0.53781818],
       [0.96969697, 0.232     , 0.99035897, 0.99035897, 0.9566    ],
       [0.96969697, 0.232     , 0.9566    , 0.9566    , 0.67133333],
       [0.96969697, 0.232     , 0.96969697, 0.96969697, 0.68771429],
       [0.99035897, 0.232     , 0.89066667, 0.99035897, 0.53781818],
       [0.96969697, 0.232     , 0.96969697, 0.9935    , 0.9566    ],
       [0.96969697, 0.232     , 0.97317647, 0.9566    , 0.53781818],
       [0.96969697, 0.232     , 0.99035897, 0.96969697, 0.68771429]])

# Compute nasal vs oral on P1 amplitude
  - load oral and nasal for awa and unaware ERPs
  - extract P1 electrodes and time window (90-120 ms)
  - compute Anovas